# Homework and bake-off: Relation extraction using distant supervision

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"
%cd /content/drive/MyDrive/StanfordCS224U/Code/cs224u-github

/content/drive/MyDrive/StanfordCS224U/Code/cs224u-github


## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [4]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [5]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [6]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [7]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [8]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [9]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [10]:
splits

{'all': Corpus with 331,696 examples; KB with 45,884 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples}

## Baselines

### Hand-build feature functions

In [10]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [11]:
featurizers = [simple_bag_of_words_featurizer]

In [12]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [ ]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.863      0.388      0.693        340       5716
author                    0.779      0.507      0.704        509       5885
capital                   0.719      0.242      0.516         95       5471
contains                  0.797      0.593      0.746       3904       9280
film_performance          0.765      0.561      0.713        766       6142
founders                  0.788      0.411      0.666        380       5756
genre                     0.569      0.171      0.388        170       5546
has_sibling               0.845      0.240      0.562        499       5875
has_spouse                0.863      0.318      0.643        594       5970
is_a                      0.659      0.217      0.468        497       5873
nationality               0.592      0.203      0.428        301       5677
parents     

Studying model weights might yield insights:

In [ ]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.516 Córdoba
     2.502 Taluks
     2.488 Valais
     ..... .....
    -1.260 joined
    -1.503 who
    -1.528 he

Highest and lowest feature weights for relation author:

     2.415 argued
     2.358 books
     2.270 writer
     ..... .....
    -3.038 1945
    -3.703 17th
    -4.758 dystopian

Highest and lowest feature weights for relation capital:

     3.736 capital
     1.893 especially
     1.768 km
     ..... .....
    -1.172 South
    -1.195 Its
    -2.042 Madras

Highest and lowest feature weights for relation contains:

     2.733 third-largest
     2.522 bordered
     2.031 transferred
     ..... .....
    -2.710 13
    -2.775 Uzbekistan
    -6.039 Bronx

Highest and lowest feature weights for relation film_performance:

     4.273 co-starring
     3.915 starring
     3.841 opposite
     ..... .....
    -1.773 comedian
    -1.872 degree
    -2.388 then

Highest and lowest feature weights for relation founders:

 

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [ ]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [ ]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [ ]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [ ]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.841      0.435      0.709        340       5716
author                    0.849      0.418      0.704        509       5885
capital                   0.759      0.232      0.521         95       5471
contains                  0.656      0.422      0.590       3904       9280
film_performance          0.834      0.329      0.638        766       6142
founders                  0.728      0.239      0.517        380       5756
genre                     0.407      0.065      0.198        170       5546
has_sibling               0.808      0.244      0.553        499       5875
has_spouse                0.859      0.348      0.664        594       5970
is_a                      0.688      0.155      0.407        497       5873
nationality               0.615      0.213      0.446        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [ ]:
from sklearn.svm import SVC

def run_svm_model_factory():

    ##### YOUR CODE HERE
    res = rel_ext.experiment(
        splits,
        featurizers=[glove_middle_featurizer],
        vectorize=False,
        verbose=True,
        model_factory=(lambda: SVC(kernel='linear'))
    )
    return res


In [ ]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [ ]:
'''
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter
'''


def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1


    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
res = rel_ext.experiment(
    splits,
    featurizers=[directional_bag_of_words_featurizer],
)



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.879      0.406      0.713        340       5716
author                    0.831      0.570      0.761        509       5885
capital                   0.806      0.263      0.571         95       5471
contains                  0.802      0.673      0.772       3904       9280
film_performance          0.820      0.659      0.782        766       6142
founders                  0.835      0.413      0.693        380       5756
genre                     0.787      0.282      0.580        170       5546
has_sibling               0.857      0.240      0.567        499       5875
has_spouse                0.888      0.348      0.678        594       5970
is_a                      0.779      0.241      0.539        497       5873
nationality               0.603      0.233      0.458        301       5677
parents     

In [ ]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [ ]:
kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
corpus.get_examples_for_entities(kbt.obj, kbt.sbj)

[Example(entity_1='xkcd', entity_2='Randall_Munroe', left='xkcd - Wikiquote xkcd From Wikiquote Jump to : navigation , search', mention_1='xkcd', middle='is a webcomic created by', mention_2='Randall Munroe', right=", a former contractor for NASA . He describes it as `` a webcomic of romance , sarcasm , math , and language . '' Munroe states there is no particular", left_POS='xkcd/NN -/: Wikiquote/NNP xkcd/VBD From/IN Wikiquote/NNP Jump/NN to/TO :/: navigation/NN ,/, search/NN', mention_1_POS='xkcd/NN', middle_POS='is/VBZ a/DT webcomic/JJ created/VBN by/IN', mention_2_POS='Randall/NNP Munroe/NNP', right_POS=",/, a/DT former/JJ contractor/NN for/IN NASA/NNP ./. He/PRP describes/VBZ it/PRP as/IN ``/`` a/DT webcomic/JJ of/IN romance/NN ,/, sarcasm/NN ,/, math/NN ,/, and/CC language/NN ./. ''/'' Munroe/NNP states/VBZ there/EX is/VBZ no/DT particular/JJ")]

In [ ]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for bg in get_tag_bigrams(ex.middle_POS):
            feature_counter[bg] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for bg in get_tag_bigrams(ex.middle_POS):
            feature_counter[bg] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    ##### YOUR CODE HERE
    prev_tag = start_symbol
    tags = get_tags(s)
    res = []
    for tag in tags:
        res.append(prev_tag + " " + tag)
        prev_tag = tag
    res.append(prev_tag + " " + end_symbol)
    return res




def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
#     print(lem)
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
# #### YOUR CODE HERE
res = rel_ext.experiment(
    splits,
    featurizers=[middle_bigram_pos_tag_featurizer],
)



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.849      0.347      0.658        340       5716
author                    0.683      0.330      0.563        509       5885
capital                   0.529      0.189      0.390         95       5471
contains                  0.743      0.588      0.706       3904       9280
film_performance          0.694      0.446      0.625        766       6142
founders                  0.496      0.168      0.357        380       5756
genre                     0.527      0.171      0.372        170       5546
has_sibling               0.638      0.162      0.402        499       5875
has_spouse                0.767      0.266      0.557        594       5970
is_a                      0.519      0.163      0.361        497       5873
nationality               0.356      0.070      0.196        301       5677
parents     

In [ ]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [ ]:
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

def synset_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for synset in get_synsets(ex.middle_POS):
            feature_counter[str(synset)] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for synset in get_synsets(ex.middle_POS):
            feature_counter[str(synset)] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]

    ##### YOUR CODE HERE
    res = []
    for pair in wt:
        word = pair[0]
        pos = convert_tag(pair[1])
        res.extend(wn.synsets(word, pos=pos))
    return res




def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
res = rel_ext.experiment(
    splits,
    featurizers=[synset_featurizer],
)




[nltk_data] Downloading package wordnet to D:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
d:\anaconda\envs\nlu\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
d:\anaconda\envs\nlu\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.786      0.335      0.620        340       5716
author                    0.772      0.426      0.664        509       5885
capital                   0.595      0.232      0.453         95       5471
contains                  0.783      0.587      0.734       3904       9280
film_performance          0.748      0.550      0.697        766       6142
founders                  0.738      0.379      0.621        380       5756
genre                     0.500      0.224      0.401        170       5546
has_sibling               0.825      0.208      0.518        499       5875
has_spouse                0.839      0.308      0.624        594       5970
is_a                      0.591      0.221      0.443        497       5873
nationality               0.480      0.163      0.346        301       5677
parents     

In [ ]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

[Synset('be.v.01'), Synset('be.v.02'), Synset('be.v.03'), Synset('exist.v.01'), Synset('be.v.05'), Synset('equal.v.01'), Synset('constitute.v.01'), Synset('be.v.08'), Synset('embody.v.02'), Synset('be.v.10'), Synset('be.v.11'), Synset('be.v.12'), Synset('cost.v.01'), Synset('angstrom.n.01'), Synset('vitamin_a.n.01'), Synset('deoxyadenosine_monophosphate.n.01'), Synset('adenine.n.01'), Synset('ampere.n.02'), Synset('a.n.06'), Synset('a.n.07'), Synset('make.v.03'), Synset('create.v.02'), Synset('create.v.03'), Synset('create.v.04'), Synset('create.v.05'), Synset('produce.v.02'), Synset('by.r.01'), Synset('aside.r.06')]


### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# STOP COMMENT: Please do not remove this comment.


In [20]:
!pip install transformers
import torch
from transformers import BertModel, BertTokenizer
import vsm

# Instantiate a Bert model and tokenizer based on `bert_weights_name`:
bert_weights_name = 'bert-base-uncased'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
##### YOUR CODE HERE
bert_model = BertModel.from_pretrained(bert_weights_name).to(device)
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)

cuda:0


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
def bert_phi(text):
    input_ids = bert_tokenizer.encode(text, add_special_tokens=True)
    # print(input_ids)
    X = torch.tensor([input_ids]).to(device)
    with torch.no_grad():
        reps = bert_model(X)
        res = reps.last_hidden_state.squeeze(0).detach().cpu().numpy()
        return res


def bert_classifier_phi(text):
    reps = bert_phi(text)[0]
    #return reps.mean(axis=0)  # Another good, easy option.
    # print(reps.shape)
    return reps


def bert_middle_featurizer(kbt, corpus, np_func=np.mean):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        rep = bert_classifier_phi(ex.middle)
        reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    # print(len(reps), reps[0].shape)
    if len(reps) == 0:
        dim = 768
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [26]:
bert_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[bert_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.287      0.097      0.206        340       5716
author                    0.780      0.418      0.665        509       5885
capital                   0.082      0.042      0.069         95       5471
contains                  0.591      0.563      0.585       3904       9280
film_performance          0.675      0.377      0.583        766       6142
founders                  0.632      0.189      0.431        380       5756
genre                     0.525      0.124      0.318        170       5546
has_sibling               0.625      0.160      0.396        499       5875
has_spouse                0.697      0.247      0.511        594       5970
is_a                      0.576      0.175      0.395        497       5873
nationality               0.586      0.249      0.461        301       5677
parents     

In [11]:
'''
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter
'''


def directional_bigram_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        prev_word = "<s>"
        for word in ex.middle.split(' ') + ["</s>"]:
            feature_counter[prev_word+" " +word+subject_object_suffix] += 1
            prev_word = word
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        prev_word = "<s>"
        for word in ex.middle.split(' ') + ["</s>"]:
            feature_counter[prev_word+" " +word+object_subject_suffix] += 1
            prev_word = word


    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
res = rel_ext.experiment(
    splits,
    featurizers=[directional_bigram_featurizer],
)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.848      0.412      0.700        340       5716
author                    0.884      0.627      0.817        509       5885
capital                   0.579      0.232      0.445         95       5471
contains                  0.832      0.676      0.795       3904       9280
film_performance          0.899      0.659      0.838        766       6142
founders                  0.865      0.389      0.695        380       5756
genre                     0.821      0.324      0.628        170       5546
has_sibling               0.857      0.240      0.567        499       5875
has_spouse                0.917      0.335      0.681        594       5970
is_a                      0.760      0.256      0.545        497       5873
nationality               0.706      0.279      0.541        301       5677
parents     

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE


In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
